In [5]:
import os
import matplotlib.pyplot as plt
import cv2
from IPython.display import Video
import json
import numpy as np


replica_root = '/datasets01/replica/061819/18_scenes'
habitat_semantic_json = os.path.join(replica_root, 'apartment_0', 'habitat', 'info_semantic.json')
#         habitat_semantic_json = os.path.join(self.sjr, scene + '_info_semantic.json')
#         print(f"Using habitat semantic json {habitat_semantic_json}")
with open(habitat_semantic_json, "r") as f:
    hsd = json.load(f)
    
# print(hsd['classes'])
id_to_class_dict = {}
for x in hsd['classes']:
    id_to_class_dict[x['id']] = x['name']


def get_maximal_relevant_id_in_view(x, traj_path):
    image_id = int(x.split('.')[0])
#     print(f'image_id {image_id}')
    
    seg = os.path.join(traj_path, 'seg')
    prop_path = os.path.join(seg, "{:05d}.npy".format(image_id))
    annot = np.load(prop_path).astype(np.uint32)
    
    ans = -1
    maxarea = 0
    for x in np.unique(annot):
        if x in ids_we_care_about:
            binary_mask = (annot == x).astype(np.int32)
            area = binary_mask.sum()
            
            # check if mask is in view
            if binary_mask[:10,:].any() or binary_mask[:,:10].any() or binary_mask[:,-10:].any() or binary_mask[-10:,:].any():
                continue
            
            if area > maxarea and area > 1000:
                maxarea = area
                ans = x
    return ans
    
    
from shutil import copyfile, rmtree
import json
import os
import matplotlib.pyplot as plt
import cv2

def acopyjson(a, b, r):
    with open(a, "r") as f:
        dat = json.load(f)
        
    dat2 = {}
    
    if os.path.isfile(b):
        with open(b, "r") as f:
            dat2 = json.load(f)
    
    ctr = 0
    for x in range(r[0], r[1]+1):
        dat2.update({ctr: dat[str(x)]})
        ctr += 1
        
    with open(b, "w") as fp:
        json.dump(dat2, fp)


def acopydir(a, b, ext, r):
#     print(f'acopydir {r}')
    if not os.path.isdir(b):
        os.makedirs(b)    
    ctr = 0
    for x in range(r[0], r[1]+1):
        f = "{:05d}{}".format(x, ext)
#         print(f)
        copyfile(os.path.join(a,f), os.path.join(b,"{:05d}{}".format(ctr, ext)))
        ctr += 1


from collections import defaultdict
    
def is_instance_in_view(traj_path, instance, img_id):
    seg = os.path.join(traj_path, 'seg')
    prop_path = os.path.join(seg, "{:05d}.npy".format(img_id))
    annot = np.load(prop_path).astype(np.uint32)
    
    if instance in np.unique(annot):
        mask = (annot == instance).astype(np.int32)
        if mask.sum() > 1000:
            return True
        
    return False
    
def vis(img_dir):
    # view 50 images from traj_out
    imgs = os.listdir(img_dir)
    n = len(imgs) 
    ctr = 0
    rows = n/5
    for x in range(int(rows)):
        fig, axs = plt.subplots(1, 5, figsize=(10,4), dpi=80)
        for y in range(5):
            axs[y].imshow(cv2.cvtColor(cv2.imread(os.path.join(img_dir, imgs[ctr])), cv2.COLOR_BGR2RGB))
            axs[y].set_title(imgs[ctr])
            ctr+=1
        plt.show()
      
def save_traj(start, end, traj_dir, inst_id, traj_out):
    print(f'save_traj {start, end}, instance_id {inst_id}, traj_out {traj_out}')
    
    # copy rgb, seg, depth and data.json
    acopydir(os.path.join(traj_dir, 'rgb'), os.path.join(traj_out, 'rgb'), '.jpg', (start,end))
    acopydir(os.path.join(traj_dir, 'depth'), os.path.join(traj_out, 'depth'), '.npy', (start,end))
    acopydir(os.path.join(traj_dir, 'seg'), os.path.join(traj_out, 'seg'), '.npy', (start,end))
    acopyjson(os.path.join(traj_dir, 'data.json'), os.path.join(traj_out, 'data.json'), (start,end))   


    
def mine_traj(data_dir, traj, pref, out_dir, min_traj_length, overall_count):
    # mine all trajs for this instance
    traj_path = os.path.join(data_dir, str(traj), pref)
    if os.path.isdir(traj_path):
        rgb = os.path.join(traj_path, 'rgb')
        imgs = os.listdir(rgb)
        
        maximal_id = []
        # 
        for x in range(len(imgs)):
            ncur = get_maximal_relevant_id_in_view(imgs[x], traj_path)
            maximal_id.append(ncur) # neighbors might not be in full view and that's okay
        
        assert len(maximal_id) == len(imgs)
        
        maximal_id = np.asarray(maximal_id)
        
        for instance in ids_we_care_about:
            ictr = 1
            x = 0
            while x < len(imgs):
                if maximal_id[x] == instance:
                    # copy this over as an instance
                    start = x
                    end = x
                    while end < len(maximal_id)-1 and is_instance_in_view(traj_path, instance, end+1):
                        end += 1

                    while start > 0 and is_instance_in_view(traj_path, instance, start-1):
                        start -= 1
                    
                    x = end
                    if end - start > min_traj_length: # min frame length
                        save_traj(start, end, traj_path, instance, os.path.join(out_dir, str(traj), str(instance), str(ictr)))
                        ictr += 1
                    
                x += 1
                              
            print(f'mined {ictr-1} trajectories for instance {instance}')
            overall_count[instance] += ictr-1
    
def prep(start_traj, num_traj, data_dir, out_dir, min_traj_length, pref=''):
    if os.path.isdir(out_dir):
        print(f'rmtree {out_dir}')
        rmtree(out_dir)
        
    os.makedirs(out_dir)
    overall_count = {}
    for iid in [243,404,196,133,166,129,170,172]:
        overall_count[iid] = 0
        
    
    for traj in range(start_traj, start_traj+num_traj): 
        print(f'mining {traj} ...')
        mine_traj(data_dir, traj, pref, out_dir, min_traj_length, overall_count)
        
    print(overall_count)
        

In [6]:
ids_we_care_about = [243,404,196,133,166,129,170,172] #[404,196,133] #[404,196,172,243,133,129,170]
min_traj_length = 20
dd = '/checkpoint/apratik/data/data/apartment_0/default/no_noise/mul_traj_200_combined_2'
od = '/checkpoint/apratik/data/data/apartment_0/default/no_noise/instance_trajectories_1116_cvprtest'
prep(10, 15, dd, od, min_traj_length)

# /checkpoint/apratik/data/data/apartment_0/default/no_noise/instance_detection_ids_allinone_test_2
# /checkpoint/apratik/data_devfair0187/apartment_0/straightline/no_noise/instance_detection_ids_allinone_auto_test_2

rmtree /checkpoint/apratik/data/data/apartment_0/default/no_noise/instance_trajectories_1116_cvprtest
mining 10 ...
save_traj (580, 619), instance_id 243, traj_out /checkpoint/apratik/data/data/apartment_0/default/no_noise/instance_trajectories_1116_cvprtest/10/243/1
save_traj (1457, 1657), instance_id 243, traj_out /checkpoint/apratik/data/data/apartment_0/default/no_noise/instance_trajectories_1116_cvprtest/10/243/2
mined 2 trajectories for instance 243
save_traj (802, 823), instance_id 404, traj_out /checkpoint/apratik/data/data/apartment_0/default/no_noise/instance_trajectories_1116_cvprtest/10/404/1
save_traj (882, 935), instance_id 404, traj_out /checkpoint/apratik/data/data/apartment_0/default/no_noise/instance_trajectories_1116_cvprtest/10/404/2
save_traj (1343, 1491), instance_id 404, traj_out /checkpoint/apratik/data/data/apartment_0/default/no_noise/instance_trajectories_1116_cvprtest/10/404/3
save_traj (2211, 2301), instance_id 404, traj_out /checkpoint/apratik/data/data/ap

KeyboardInterrupt: 

<img src=vid_path align="center">

In [21]:
active_data_dir = '/checkpoint/apratik/data_devfair0187/apartment_0/straightline/no_noise/1633991019'
active_out_dir = '/checkpoint/apratik/data_devfair0187/apartment_0/straightline/no_noise/instance_trajectories'
prep(5, active_data_dir, active_out_dir, min_traj_length, pref='default')

rmtree /checkpoint/apratik/data_devfair0187/apartment_0/straightline/no_noise/instance_trajectories
save_traj (0, 33), instance_id 243, traj_out /checkpoint/apratik/data_devfair0187/apartment_0/straightline/no_noise/instance_trajectories/1/243/1
save_traj (110, 190), instance_id 243, traj_out /checkpoint/apratik/data_devfair0187/apartment_0/straightline/no_noise/instance_trajectories/1/243/2
mined 2 trajectories for instance 243
save_traj (37, 104), instance_id 404, traj_out /checkpoint/apratik/data_devfair0187/apartment_0/straightline/no_noise/instance_trajectories/1/404/1
save_traj (1111, 1217), instance_id 404, traj_out /checkpoint/apratik/data_devfair0187/apartment_0/straightline/no_noise/instance_trajectories/1/404/2
save_traj (1473, 1503), instance_id 404, traj_out /checkpoint/apratik/data_devfair0187/apartment_0/straightline/no_noise/instance_trajectories/1/404/3
mined 3 trajectories for instance 404
save_traj (1194, 1227), instance_id 196, traj_out /checkpoint/apratik/data_devf